In [48]:
# coding: utf8
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
import unicodedata

class ScrapeWTAStats(object):
    
    def __init__(self):
        self.players_game_data = []
        self.players_stat_data = []
        
    def __call__(self):
        for self.i in range(101):
            self.go_on_wta_ranking_page()
            self.go_on_player_profile_page(self.browser)
            self.go_on_player_statistics_page(self.browser)
            self.scrape_player_stat_data(self.browser, self.players_stat_data)
            self.go_on_player_game_data_page(self.browser)
            self.scrape_player_game_data(self.browser, self.players_game_data)
            self.data_to_csv(self.players_stat_data, self.players_game_data)
            
    # --- Going on the core tennis website ---
    def go_on_wta_ranking_page(self):
        url = "http://www.coretennis.net/"
        delay = 30 # seconds

        while True:
            try:
                browser = webdriver.Firefox("C:\Users\jbadiabo\PycharmProjects\Sibyl\geckodriver.exe")
                browser.set_page_load_timeout(delay)
                browser.get(url)
                # ---Going on the WTA Ranking page---
                browser.maximize_window()

                try: 
                    tennis_ranking_button = browser.find_element_by_xpath(".//div[@class='menu1']/a[4]")
                except NoSuchElementException:
                    tennis_ranking_button = browser.find_element_by_partial_link_text("Tennis Rankings")
                tennis_ranking_button.click()
                browser.get(browser.current_url)
                
                try: 
                    wta_ranking_button = browser.find_element_by_xpath(".//div[@class='menu2']/a[2]")
                except NoSuchElementException:
                    wta_ranking_button = browser.find_element_by_link_text("WTA Rankings")
                wta_ranking_button.click()
                browser.get(browser.current_url)
                
            except TimeoutException:
                print "Timeout, retrying1.."
                browser.quit()
                delay += 10
                continue
            else:
                break

        # ---Finding the number of players to create a variable to iterate with---
        try:
            table = browser.find_element_by_id('rtable2')
        except NoSuchElementException:
            table = browser.find_elements_by_tag_name("table")[0]

        body = table.find_element_by_tag_name('tbody')
        table_rows = body.find_elements_by_tag_name('tr')
        player_number_list = len(table_rows)

        self.browser = browser
        
    def go_on_player_profile_page(self, browser):
        # ---Going on player profile---
        current_url = browser.current_url
        delay = 30
        while True:
            try:
                try:
                    table = browser.find_element_by_id('rtable2')
                except NoSuchElementException:
                    table = browser.find_elements_by_tag_name("table")[0]

                body = table.find_element_by_tag_name('tbody')
                table_rows = body.find_elements_by_tag_name('tr')
                table_rows = table_rows[1:]
                player_name_cell = table_rows[self.i].find_elements_by_tag_name('td')[1]
                player_profile_link = player_name_cell.find_element_by_tag_name('a')            
                player_profile_link.click()
                browser.get(browser.current_url)
            except TimeoutException:
                while True:
                    try:        
                        print "Timeout, retrying4.."
                        browser.quit()
                        browser = webdriver.Chrome("C:\Users\jbadiabo\PycharmProjects\Sibyl\chromedriver.exe")
                        delay += 10
                        browser.set_page_load_timeout(delay)
                        browser.get(current_url)
                        browser.maximize_window()
                    except TimeoutException:
                        continue
                    else:
                        break  
            # For the big try
                continue
            else:
                break

        self.browser = browser

    # ---Going on player statistics profile---
    def go_on_player_statistics_page(self, browser):
        current_url = browser.current_url
        delay = 30
        
        while True:
            try:
                try:
                    player_statistics_link = browser.find_element_by_link_text("Statistics")
                except NoSuchElementException:
                    player_statistics_link = browser.find_element_by_xpath(".//div[@class='ppNav']/a[4]")

                player_statistics_link.click()
                browser.get(browser.current_url)
            except TimeoutException:
                while True:
                    try:        
                        print "Timeout, retrying4.."
                        browser.quit()
                        browser = webdriver.Chrome("C:\Users\jbadiabo\PycharmProjects\Sibyl\chromedriver.exe")
                        delay += 10
                        browser.set_page_load_timeout(delay)
                        browser.get(current_url)
                        browser.maximize_window()
                    except TimeoutException:
                        continue
                    else:
                        break         
            # For the big try
                continue
            else:
                break

        self.browser = browser
        
    # ---Scraping player stats---
    def scrape_player_stat_data(self, browser, players_stat_data):
        main_div = browser.find_element_by_id("colMainContent1b")

        player_name_header = main_div.find_element_by_class_name("ppHeader")
        player_name = player_name_header.find_element_by_tag_name('h1').text.encode("ascii", "ignore")

        player_name = player_name.rsplit(' ', 2)[0] # 2 for 2nd split starting from the right

        table_titles = main_div.find_elements_by_tag_name("h2")
        table_titles = [x.text.encode('ascii', 'ignore') for x in table_titles]
        stat_tables = main_div.find_elements_by_class_name("sTable")
        number_of_years = len(stat_tables)

        year_player_data = []

        for stat_table, table_title in zip(stat_tables, table_titles):

            try:
                year = table_title.split(" ")[0]
                # ------------------------------------------------------------------------------------
                table_footer = stat_table.find_element_by_tag_name("tfoot")
                table_footer_row = table_footer.find_element_by_tag_name("tr")
                data = [player_name, year]
                table_footer_row_data = table_footer_row.find_elements_by_tag_name("td")

                for i in table_footer_row_data:
                    data.append(i.text.encode('ascii', 'ignore'))

                year_player_data.append(data)
            except NoSuchElementException:
                continue

        players_stat_data += year_player_data
        
        self.players_stat_data = players_stat_data

        self.browser = browser
        
    # ---Going on the player game data---
    def go_on_player_game_data_page(self, browser):
        current_url = browser.current_url
        delay = 30
        while True:
            try:
                main_div = browser.find_element_by_id("colMainContent1b")
                nav_div = main_div.find_element_by_class_name("ppNav")

                try:
                    results_button = nav_div.find_element_by_partial_link_text("Results")
                except NoSuchElementException:
                    results_button = nav_div.find_element_by_xpath(".//div[@class='ppNav']/a[3]")

                results_button.click()
                browser.get(browser.current_url)
            except TimeoutException:
                while True:
                    try:        
                        print "Timeout, retrying4.."
                        browser.quit()
                        browser = webdriver.Chrome("C:\Users\jbadiabo\PycharmProjects\Sibyl\chromedriver.exe")
                        delay += 10
                        browser.set_page_load_timeout(delay)
                        browser.get(current_url)
                        browser.maximize_window()
                    except TimeoutException:
                        continue
                    else:
                        break  
            # For the big try
                continue
            else:
                break      

        self.browser = browser

    # ---Scraping player game_data---
    def scrape_player_game_data(self, browser, players_game_data):
        player_game_data = []

        year_nav_div = browser.find_element_by_class_name("shadetabs")
        year_button_links = year_nav_div.find_elements_by_tag_name("li")

        main_div = browser.find_element_by_class_name("tabcontentstyle")
        year_divs = main_div.find_elements_by_class_name("tabcontent")

        for year_button_link, year_div in zip(year_button_links, year_divs):
            year_button_link.click()

            # --- Scraping the year ---
            year = year_nav_div.find_element_by_css_selector("li.selected").text.encode("ascii", "ignore")

            # ---Scraping player name---
            header = browser.find_element_by_class_name("ppHeader")
            player_name = header.find_element_by_css_selector('h1').text.encode("ascii", "ignore")
            player_name = player_name.rsplit(' ', 2)[0] # 2 for 2nd split starting from the right

            # ---Scraping game data---
            containers = year_div.find_elements_by_class_name("pprContainer")
            year_container_rows_data = []

            for container in containers:

                container_datetime_range = container.find_element_by_css_selector("div.pprHead")
                try:
                    container_datetime_range = container_datetime_range.find_elements_by_tag_name("div")[0].text.encode("ascii", "ignore")
                except NoSuchElementException:
                    container_datetime_range = container_datetime_range.find_element_by_class_name("plM1")

                container_datetime_range = container_datetime_range.replace("\n", " - ")

                container_rows = container.find_elements_by_css_selector("div.pprRow")

                container_rows_data = []
                for row in container_rows:
                    data = row.find_elements_by_tag_name("div")
                    data = [x.text.encode("ascii", "ignore") for x in data]

                    indices = 0, 2, 4
                    data = [i for j, i in enumerate(data) if j not in indices]
                    data.insert(0, data.pop(1))
                    data.insert(0, player_name)
                    data.insert(0, container_datetime_range)
                    data.insert(0, year)


                    container_rows_data.append(data)

                year_container_rows_data = year_container_rows_data + container_rows_data

            player_game_data = player_game_data + year_container_rows_data

        players_game_data = players_game_data + player_game_data
        
        self.players_game_data = players_game_data

        browser.quit()
        
    def data_to_csv(self, players_stat_data, players_game_data):
        players_stat_data_df = pd.DataFrame(players_stat_data, 
                                            columns=['Player_name', 'Year', 'Tourn', 'Titles', 'Matches', 'Wins', 'Losses', 'PCT', '6-0', '0-6', '7-6', '6-7'])
        players_stat_data_df['Player_name'] = players_stat_data_df['Player_name'].str.rstrip().str.lstrip()

        # ------------------------------------------------------------

        players_game_data_df = pd.DataFrame(players_game_data, 
                                            columns=['Year', 'Week', 'Player_A', 'Player_B', 'True_Result'])
        players_game_data_df['Player_B'] = players_game_data_df['Player_B'].str[:-5]
        players_game_data_df.insert(1, 'Start', players_game_data_df['Week'].str.split(' - ').str.get(0))
        players_game_data_df.drop('Week', axis = 1, inplace = True)
        players_game_data_df.insert(1, 'Week', players_game_data_df['Year'] + ',' + players_game_data_df['Start'])
        players_game_data_df.drop('Start', axis = 1, inplace = True)
        players_game_data_df['Week'] = players_game_data_df['Week'].str.replace(' ', ',')
        players_game_data_df['Week'] = pd.to_datetime(players_game_data_df['Week'], infer_datetime_format=True)
        players_game_data_df['Player_A'] = players_game_data_df['Player_A'].str.rstrip().str.lstrip()
        players_game_data_df['Player_B'] = players_game_data_df['Player_B'].str.rstrip().str.lstrip()

        # Dropping duplicate rows
        players_game_data_df['Winner'] = players_game_data_df.apply(lambda x: x['Player_A'] if x['True_Result'] == 'W' else x['Player_B'], axis = 1)
        players_game_data_df['Looser'] = players_game_data_df.apply(lambda x: x['Player_A'] if x['True_Result'] == 'L' else x['Player_B'], axis = 1)
        players_game_data_df.drop_duplicates(subset=['Week', 'Winner', 'Looser'], inplace=True)
        players_game_data_df.drop(['Winner', 'Looser'], axis = 1, inplace = True)

        # -----------------------------------------------------
        players_stat_data_df.to_csv("wta_player_stats.csv", mode='w+')

        players_game_data_df['Week'].to_csv("date_wta_game_stats.csv", mode='w+', header=False, index=False)
        players_game_data_df.to_csv("wta_game_stats.csv", index=False, mode='w+')

        self.players_stat_data_df = players_stat_data_df
        self.players_game_data_df = players_game_data_df

In [49]:
w = ScrapeWTAStats()
w()

WindowsError: [Error 267] The directory name is invalid: 'C:\\Users\\jbadiabo\\PycharmProjects\\Sibyl\\geckodriver.exe/*.*'

In [52]:
range(0, 11)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [53]:
range(11, 21)

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [54]:
range(91, 101)

[91, 92, 93, 94, 95, 96, 97, 98, 99, 100]